# Variationist Example 2: Custom Metrics

**Welcome! In this example guide, you will be shown how to use your own custom metrics inside 🕵️‍♀️ Variationist.**


In [1]:
# Install using pip
!pip install variationist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from variationist import Inspector, InspectorArgs, Visualizer, VisualizerArgs

Load your own dataset in .tsv or .csv format, or use a dataset that is available on HuggingFace


In [3]:
# my_dataset = "my_dataset.tsv"
my_dataset = "hf::truthful_qa::generation::validation"

Let's use a metric of our own in addition to the built-in ones of Variationist.

Here is a metric that counts how often questions contain at least one of the 5 W-words (what, where, who, why, when) in the questions of each category of the `truthful_qa` dataset.

In [4]:
def count_5w(label_values_dict, subsets_of_interest, inspector_args):
    results_dict = {}
    # for each variable we are analyzing
    for label in label_values_dict:
        results_dict[label] = {}
        # we loop through each value that variable can take
        for i in range(len(label_values_dict[label])):
            results_dict[label][label_values_dict[label][i]] = 0
            # we loop through each text in the subset we are currently analyzing
            for text in subsets_of_interest[label][i]:
                for w in ["what", "where", "who", "why", "when"]:
                    if w in text:
                        results_dict[label][label_values_dict[label][i]] += 1

    return results_dict

In [5]:
# Define the inspector arguments. For now, let's just select a handful of metrics.
inspector_args = InspectorArgs(
		 text_names=["question"],
		 var_names=["category"],
		 metrics=["pmi", "stats", "npw_relevance", "ttr", "freq", count_5w],
		 language="en",
		 n_tokens=1,
		 n_cooc=1
)

In [6]:
# Create an inspector instance, run it, and get the results in json
results = Inspector(dataset=my_dataset, args=inspector_args).inspect()

INFO: No values have been set for var_types. Defaults to nominal.
INFO: No values have been set for var_semantics. Defaults to general.
INFO: The metadata we will be using for the current analysis are:
{'text_names': ['question'], 'var_names': ['category'], 'metrics': ['pmi', 'stats', 'npw_relevance', 'ttr', 'freq', 'count_5w'], 'var_types': ['nominal'], 'var_semantics': ['general'], 'var_subsets': None, 'var_bins': [0], 'tokenizer': 'whitespace', 'language': 'en', 'n_tokens': 1, 'n_cooc': 1, 'unique_cooc': False, 'cooc_window_size': 0, 'freq_cutoff': 3, 'stopwords': False, 'custom_stopwords': None, 'lowercase': False, 'ignore_null_var': False}
INFO: all column identifiers are treated as column names.
INFO: 'Loading hf::truthful_qa::generation::validation' as a HuggingFace dataset. We assume the third element in the specified string is the subset ("generation") and the last is the split ("validation").


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

INFO: Tokenizing the question column...


100%|██████████| 817/817 [00:00<00:00, 16718.53it/s]


...

INFO: Currently calculating metric: 'ttr'


100%|██████████| 38/38 [00:00<00:00, 4173.87it/s]


INFO: Currently calculating metric: 'freq'


100%|██████████| 38/38 [00:00<00:00, 6293.77it/s]

INFO: Currently calculating metric: 'count_5w'


In [7]:
# Define the visualizer arguments
visualizer_args = VisualizerArgs(
	output_folder="output", zoomable=True, ngrams=None, output_formats=["html", "png"])

# Create dynamic visualizations of the results
charts = Visualizer(input_json=results, args=visualizer_args).create()

Reading json data...
INFO: Creating a BarChart object for metric "pmi"...
INFO: Saving it to the filepath: "output/pmi/BarChart.html".
INFO: Saving it to the filepath: "output/pmi/BarChart.png".
INFO: Creating a BarChart object for metric "stats"...
INFO: Saving it to the filepath: "output/stats/StatsBarChart.html".
INFO: Saving it to the filepath: "output/stats/StatsBarChart.png".
INFO: Creating a BarChart object for metric "npw_relevance"...
INFO: Saving it to the filepath: "output/npw_relevance/BarChart.html".
INFO: Saving it to the filepath: "output/npw_relevance/BarChart.png".
INFO: Creating a BarChart object for metric "ttr"...
INFO: Saving it to the filepath: "output/ttr/DiversityBarChart.html".
INFO: Saving it to the filepath: "output/ttr/DiversityBarChart.png".
INFO: Creating a BarChart object for metric "freq"...
INFO: Saving it to the filepath: "output/freq/BarChart.html".
INFO: Saving it to the filepath: "output/freq/BarChart.png".
INFO: Creating a BarChart object for metri

In [8]:
charts["count_5w"]["BarChart"]

alt.Chart(...)